In [25]:
PATH='/home/luis/sustituciones/centros/actual/Moodle/rubricas'
comps=['obs',
       'ccl',
       'cmcct',
       'cd',
       'caa',
       'csc',
       'csiee',
       'ccec'
    ]
cursos=[
    '1a',
    '1b',
    '1c',
    '3a',
    '3b',
    '3c',
    '3d'
]

info={'1a':[22,'Xuño','Titora','MªLourdes Manteiga Gorgal'],'1b':[22,'Xuño','Titora','MªCarmen Pereiro Vizcaino'],'1c':[22,'Xuño','Titor','Ramón Iglesias Veiga'],'3a':[23,'Xuño','Titor','Francisco Domínguez Gonzalez'],'3b':[23,'Xuño','Titor','J. Ramón Rey Rodríguez'],'3c':[23,'Xuño','Titora','Moniqe Otero López'],'3d':[23,'Xuño','Titora','Araceli Andrade Fernández']}



In [26]:
def resuelve(vals):
    pos=range(len(vals))
    return round(1.0*sum([u*v for u,v in zip(vals,pos)])/sum(vals))

In [27]:
def nomtohtml(n):
    '''
    converts utf names to html names
    '''
    from re import sub
    
    acute=[
        ['á','a'],['Á','A'],['é','e'],['É','E'],['í','i'],['Í','I'],['ó','o'],['Ó','O'],['ú','u'],['Ú','U']
    ]
    tilde=[
        ['ñ','n'],['Ñ','N']
    ]
    #print n
    for item in acute:
        n=sub(item[0],'&'+item[1]+'acute;',n)
    #print n
    for item in tilde:
        n=sub(item[0],'&'+item[1]+'tilde;',n)
    #print n
    return n

In [28]:
def import_comp(nomfich):
    '''
    importa los resultados de la encuesta de competencias
    devuelve un diccionario donde:
        key-> nombre alumno
        value-> lista con nº votos por nivel
    '''
    import re
    
    try:
        fich=open(nomfich+'.csv.0','r')
        cont=fich.readlines()
        fich.close()
    except:
        print 'No se encuentra el fichero %s' % nomfich+'.csv.0'
        return {}

    data=[]
    quest=[]
    nump=0
    ok=False
    count=0

    #Load data
    for aux in cont:
        item= aux.strip().split(',')
        if 'Encuestas:' in item[0]:
            nump=item[0].split(':')[1]
            nump=int(nump.strip('"'))
        elif 'Pregunta' in item[0] and 'Respuesta' in item[1]:
            ok=True
        elif ok:
            if not count % 3:
                cad=item[0].strip('"')+','+item[1].strip('"')
                temp=[re.sub(' ,',',',cad)]
                if count<1:
                    quest=[dummy.strip('"') for dummy in item[2:]]
            elif count %3 ==1:
                temp+=[int(dummy) for dummy in item[1:]]
                data.append(temp)
            count+=1
    
    #Process data
    proces=[]
    for item in data:
        nom=item[0]
        temp=item[1:-1]
        temp.reverse()
        mx=max(temp)
        temp=map(lambda x: x//mx, temp)
        if sum(temp)>1:
            pos=resuelve(temp)
            for index in range(len(temp)):
                if not index==int(pos):
                    temp[index]=0
                else:
                    temp[index]=1
        temp.insert(0,nom)
        proces.append(temp)
    return proces


In [29]:
def informe_competencias(curso,nome,resp,dia,mes,tratam,tutor,nombre):
    fich=open(PATH+'/plantilla_informe.html')
    aux=fich.readlines()
    fich.close()
    base=''.join(aux)
    if '1' in curso:
        fich=open(PATH+'/plantilla_orientac1eso.html')
        aux=fich.readlines()
        orientaeso=''.join(aux)
    else:
        fich=open(PATH+'/plantilla_orientac3eso.html')
        aux=fich.readlines()
        orientaeso=''.join(aux)
   
    relleno=[]
    relleno.append(nome)
    relleno.append(nome)
    relleno+=resp
    relleno.append(orientaeso)
    relleno.append(dia)
    relleno.append(mes)
    relleno.append(tratam)
    relleno.append('D. '+tutor)
    relleno.append('<textarea rows=10 cols=80 name="obs" wrap=virtual placeholder=""></textarea>')
    fich=open(PATH+'/informes/'+nombre+'.html','w')
    fich.write(base % tuple(relleno))
    fich.close()

In [44]:
import os
for curso in cursos:
    #curso='/informes/2-ESO-A-'
    for comp in comps:
        
        
        if curso=='3d' and comp=='cmcct':
            #print alumns
            for alumn in alumns.keys():
                alumns[alumn].append([0]*4)
                
        else:
            item=import_comp(PATH+'/informes/'+curso+'-'+comp)
            if comp=='obs':
                alumns_ord=[dummy[0] for dummy in item]
                alumns={dummy[0]:[dummy[1:]] for dummy in item}
            else:
                for num in item:
                    alumns[num[0]].append(num[1:])
    dia,mes,tratam,tutor=info[curso]


    for alum in alumns_ord:
        nom=nomtohtml(alum)
        res=[]
        for item in alumns[alum]:
            res+=item
        resp=map(lambda x: 'checked' if x else '', res)
        informe_competencias(curso,nom,resp,dia,mes,tratam,tutor,alum)
    cad='zip -j %s/informes/%s %s/informes/*.html; rm -f %s/informes/*.html' %(PATH,curso,PATH,PATH)
    os.system(cad)